In [9]:
import sqlite3
import pandas as pd
from IPython.core.display import HTML, Markdown

In [10]:
class Report:
    def __init__(self):
        self.md = ""

    def addTitle(self, title):
        self.md += f"# {title}\n\n"

    def addSubTitle(self, title):
        self.md += f"## {title}\n\n"

    def addParagraph(self, text):
        for line in text.splitlines():
            self.md += f"{line}\n"
        self.md += "\n"

    def addText(self, text):
        self.addParagraph(text)
    
    def append(self, text):
        self.md += text + "\n"

    def clear(self):
        self.md = ""

    def debug(self):
        print(self.md)

    def display(self):
        display(Markdown(self.md))

    def markdown(self):
        return self.md

    def saveToFile(self, fn):
        with open(fn, 'w') as f:
            f.write(self.md)

    def addLessons(self, e, con):
        sql = f"""SELECT
                    giorno,
                    L.data as dataISO,
                    substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data,
                    L.idFasciaOraria,
                    L.idLezione
                FROM
                    Edizione E INNER JOIN 
                    Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
                    Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
                WHERE L.idEdizione = '{e}'
                ORDER BY E.idEdizione, L.data, L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "| Incontro | Data | Ora | Incontro | Data | Ora |\n|--:|:-:|:-:|--:|:-:|:-:|\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"|{l["idLezione"]}|{l["giorno"].lower()} {l["data"]} |{l["idFasciaOraria"]}"
            if j % 2 == 0:
                str += ""
            else:
                str += "|\n"
        self.md += str + "\n\n";

    def addLessonsByStaff(self, p, con):
        sql = f"""SELECT
                rank() OVER (ORDER BY L.data, L.idFasciaOraria) AS progressivo,
            	E.idClasse,
            	giorno,
            	L.data AS dataISO,
            	L.idFasciaOraria,
            	E.idCorso ,
            	L.idLezione,
            	F.ruolo,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) as data
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
            ORDER BY
            	L.data,
            	L.idFasciaOraria"""
        ls = pd.read_sql_query(sql, con)
        str = "|Progr.| Data | Ora | Corso | Lezione | Classe |\n|--:|:-:|:-:|:-:|--:|:-:|\n"
        for j in range(len(ls)):
            l = ls.iloc[j]
            str += f"|{l["progressivo"]}|{l["giorno"].lower()} {l["data"]}|{l["idFasciaOraria"]}|{l["idCorso"]}|{l["idLezione"]}|{l["idClasse"]}|\n"    
        self.md += str + "\n\n"

    def addDailySchedule(self, p, con):
        sql = f"""SELECT
				rank() OVER (ORDER BY L.data) AS progressivo,
            	giorno,
            	substr(L.data, 9, 2) || '/' || substr(L.data, 6, 2) || '/' || substr(L.data, 1, 4) AS data,
				group_concat(L.idFasciaOraria ORDER BY L.idFasciaOraria ASC) AS ore,
				group_concat(DISTINCT E.idCorso) AS corsi
            FROM
            	Formatore F INNER JOIN
            	Personale P ON F.idPersonale=P.idPersonale INNER JOIN
            	Edizione E ON F.idEdizione=E.idEdizione INNER JOIN
            	Lezione L ON E.idEdizione=L.idEdizione INNER JOIN
            	Giorno G ON cast (strftime('%w', L.data) as integer)=G.idGiorno
            WHERE
            	P.idPersonale = '{p}'
			GROUP BY L.data
            ORDER BY
            	L.data"""
        str = "|Giornata| Data | Ore | Corsi |\n|--:|:-:|:-:|:-:|\n"
        ds = pd.read_sql_query(sql, con)
        for j in range(len(ds)):
            d = ds.iloc[j]
            str += f"|{d["progressivo"]}|{d["giorno"].lower()} {d["data"]}|{d["ore"]}|{d["corsi"]}|\n"
        self.md += str + "\n\n"

In [11]:
con = sqlite3.connect("data/calendario_corsi.sqlite")
report = Report()

In [4]:
report.clear()
report.addTitle("Calendario corsi")
for c in ['RelAx', 'CyberSec', 'CryptoWithBlocks', 'LabMat2', 'PhyComp', 'RicOp']: # , 'EduRobot', 'PassInf']:
    sql = "SELECT idEdizione, denominazione, descrizione, idClasse FROM Edizione INNER JOIN Corso USING (idCorso) WHERE Corso.idCorso='" + c + "' ORDER BY denominazione"
    edizioni = pd.read_sql_query(sql, con)
    for i in range(len(edizioni)):
        edizione = edizioni.iloc[i]
        editionRep = Report()
        # Titolo
        editionRep.addSubTitle(edizione["denominazione"]) # edizione["idEdizione"] + " - " + 
        # Studenti
        editionRep.addText(f"**Studenti/Classe**: {edizione["idClasse"]}")
        # Formatore
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Formatore'"
        formatore = pd.read_sql_query(sql, con)["idPersonale"][0]
        editionRep.addText(f"**Formatore**: *{formatore}*")
        # Tutor
        sql = f"SELECT idPersonale FROM Formatore WHERE idEdizione = '{edizione["idEdizione"]}' AND ruolo = 'Tutor'"
        tutor = pd.read_sql_query(sql, con)["idPersonale"][0]
        editionRep.addText(f"**Tutor**: *{tutor}*")
        # Descriione
        editionRep.addParagraph(f"**Descrizione**: {edizione["descrizione"]}")
        # Lezioni
        editionRep.addText(f"**Calendario**")
        editionRep.addLessons(edizione["idEdizione"], con)
        editionRep.saveToFile("./gen_docs/editions/" + edizione["idEdizione"] + ".md")
        report.append(editionRep.markdown())
report.display()

# Calendario corsi

## ALGEBRA DELLE RELAZIONI 1

**Studenti/Classe**: S5B+S5D

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|ven 13/12/2024 |5|2|ven 20/12/2024 |5|
|3|ven 10/01/2025 |5|4|ven 17/01/2025 |5|
|5|ven 24/01/2025 |5|6|ven 14/03/2025 |5|
|7|ven 21/03/2025 |5|8|ven 28/03/2025 |5|
|9|ven 04/04/2025 |5|10|ven 11/04/2025 |5|



## ALGEBRA DELLE RELAZIONI 2

**Studenti/Classe**: S4E

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 05/02/2025 |2|2|mer 12/02/2025 |2|
|3|mer 19/02/2025 |2|4|mer 26/02/2025 |2|
|5|mer 05/03/2025 |2|6|mer 12/03/2025 |2|
|7|mer 19/03/2025 |2|8|mer 26/03/2025 |2|
|9|mer 02/04/2025 |2|10|mer 09/04/2025 |2|



## ALGEBRA DELLE RELAZIONI 3

**Studenti/Classe**: S5E

**Formatore**: *Massi*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 11/12/2024 |3|2|lun 16/12/2024 |2|
|3|mer 18/12/2024 |3|4|mer 08/01/2025 |3|
|5|lun 13/01/2025 |2|6|mer 15/01/2025 |3|
|7|lun 20/01/2025 |2|8|mer 22/01/2025 |3|
|9|lun 27/01/2025 |2|10|mer 29/01/2025 |3|



## ALGEBRA DELLE RELAZIONI 4

**Studenti/Classe**: S4F

**Formatore**: *Massi*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso introduce le basi della teoria delle basi di dati relazionali, un pilastro fondamentale dell'informatica intesa come disciplina che studia i metodi per estrarre informazioni in modo automatico da dati grezzi.
La presentazione degli argomenti è declinata in modo rendere evidente come l'informatica sia una disciplina scientifica e ingegneristica. Vengono esposti sia gli aspetti scientifici e formali, come quelli dell'algebra delle relazioni, sia quelli ingegneristici legati all'efficienza computazionale degli algoritmi.
I contenuti si collocano in un contesto interdisciplinare, come è proprio dell'informatica, dove la teoria si unisce alla pratica, grazie all'utilizzo di una piattaforma web per la valutazione di espressioni algebriche relazionali.
Obiettivi generali:
  -  cogliere la stretta relazione tra pensiero scientifico e sviluppo tecnologico.
  - comprendere le strutture fondamentali dei ragionamenti logico-deduttivi e padroneggiare il linguaggio logico-formale per risolvere problemi di varia natura.
  - utilizzare strumenti informatici per modellare e risolvere problemi.
  - acquisire una comprensione approfondita della disciplina informatica per orientare le proprie scelte future, sia a livello universitario che professionale.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 11/12/2024 |4|2|lun 16/12/2024 |1|
|3|mer 18/12/2024 |4|4|mer 08/01/2025 |4|
|5|lun 13/01/2025 |1|6|mer 15/01/2025 |4|
|7|lun 20/01/2025 |1|8|mer 22/01/2025 |4|
|9|lun 27/01/2025 |1|10|mer 29/01/2025 |4|



## CYBER SICUREZZA 1

**Studenti/Classe**: S3B

**Formatore**: *Taccari*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso avvicina le studentesse e gli studenti alla sicurezza informatica tramite attività di gioco (challenge). È propedeutico alla partecipazione alle iniziative sulla sicurezza informatica proposte dalla rete CyberHighSchools (OliCyber, CyberChallenge, CyberTrials). Il laborartorio propone l'uso di un ecosistema hardware (Rasperry Pi 5) e software (sistema operativo GNU/Linux, interprete Python, strumenti per l'analisi del traffico di rete, strumenti per l'analisi dei file eseguibili) a sorgente aperto con il quale scoprire giocando le problematiche di sicurezza, sviluppare una coscienza critica sull'argomento e promuovere comportamenti orientati ad una protezione dei dati personali.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 29/01/2025 |5|2|ven 31/01/2025 |6|
|3|mer 05/02/2025 |5|4|ven 07/02/2025 |6|
|5|mer 12/02/2025 |5|6|mer 19/02/2025 |5|
|7|ven 21/02/2025 |6|8|mer 05/03/2025 |5|
|9|ven 07/03/2025 |6|10|mer 12/03/2025 |5|
|11|ven 14/03/2025 |6|12|mer 19/03/2025 |5|
|13|ven 21/03/2025 |6|14|mer 26/03/2025 |5|
|15|ven 28/03/2025 |6|16|mer 02/04/2025 |5|
|17|ven 04/04/2025 |6|18|mer 09/04/2025 |5|
|19|ven 11/04/2025 |6|20|mer 16/04/2025 |5|



## CYBER SICUREZZA 2

**Studenti/Classe**: S4B

**Formatore**: *Taccari*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso avvicina le studentesse e gli studenti alla sicurezza informatica tramite attività di gioco (challenge). È propedeutico alla partecipazione alle iniziative sulla sicurezza informatica proposte dalla rete CyberHighSchools (OliCyber, CyberChallenge, CyberTrials). Il laborartorio propone l'uso di un ecosistema hardware (Rasperry Pi 5) e software (sistema operativo GNU/Linux, interprete Python, strumenti per l'analisi del traffico di rete, strumenti per l'analisi dei file eseguibili) a sorgente aperto con il quale scoprire giocando le problematiche di sicurezza, sviluppare una coscienza critica sull'argomento e promuovere comportamenti orientati ad una protezione dei dati personali.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 29/01/2025 |6|2|ven 31/01/2025 |4|
|3|mer 05/02/2025 |6|4|ven 07/02/2025 |4|
|5|mer 12/02/2025 |6|6|mer 19/02/2025 |6|
|7|ven 21/02/2025 |4|8|mer 26/02/2025 |6|
|9|ven 28/02/2025 |4|10|mer 05/03/2025 |6|
|11|ven 07/03/2025 |4|12|mer 12/03/2025 |6|
|13|ven 14/03/2025 |4|14|mer 19/03/2025 |6|
|15|ven 21/03/2025 |4|16|mer 26/03/2025 |6|
|17|ven 28/03/2025 |4|18|mer 02/04/2025 |6|
|19|ven 04/04/2025 |4|20|mer 09/04/2025 |6|



## CRITTOGRAFIA CON BLOCCHI 1

**Studenti/Classe**: S1B

**Formatore**: *Taccari*

**Tutor**: *Sciamanna*

**Descrizione**: Il corso è un'introduzione alle "grandi idee" della crittografia. L'approccio didattico procede "per scoperta" con attività che si susseguono con una successione di crittosistemi (dai classici ai più moderni). Lo studente può sperimentare caratteristiche di ciascun crittosistema, realizzado possibili procedure di attacco e valutandone i limiti. Dalle debolezze di un sistema lo studente dovrebbe avere necessità di scoprire il crittosistema successivo, che avrà superato i limiti del sistema precedente ma che potrà presentare altri aspetti limitanti. L'impianto generale del corso si basa su M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO". I crittosistemi, la loro analisi ed le possibili procedure di attacco sono realizzate in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna).

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 05/02/2025 |1|2|sab 08/02/2025 |4|
|3|mer 12/02/2025 |1|4|sab 15/02/2025 |4|
|5|mer 19/02/2025 |1|6|sab 22/02/2025 |4|
|7|mer 26/02/2025 |1|8|sab 01/03/2025 |4|
|9|mer 05/03/2025 |1|10|sab 08/03/2025 |4|



## CRITTOGRAFIA CON BLOCCHI 2

**Studenti/Classe**: S4G

**Formatore**: *Taccari*

**Tutor**: *Lorenzoni*

**Descrizione**: Il corso è un'introduzione alle "grandi idee" della crittografia. L'approccio didattico procede "per scoperta" con attività che si susseguono con una successione di crittosistemi (dai classici ai più moderni). Lo studente può sperimentare caratteristiche di ciascun crittosistema, realizzado possibili procedure di attacco e valutandone i limiti. Dalle debolezze di un sistema lo studente dovrebbe avere necessità di scoprire il crittosistema successivo, che avrà superato i limiti del sistema precedente ma che potrà presentare altri aspetti limitanti. L'impianto generale del corso si basa su M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO". I crittosistemi, la loro analisi ed le possibili procedure di attacco sono realizzate in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna).

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mar 04/02/2025 |2|2|mar 11/02/2025 |2|
|3|mar 18/02/2025 |2|4|mar 25/02/2025 |2|
|5|mar 04/03/2025 |2|6|mar 11/03/2025 |2|
|7|mar 18/03/2025 |2|8|mar 25/03/2025 |2|
|9|mar 01/04/2025 |2|10|mar 08/04/2025 |2|



## CRITTOGRAFIA CON LINGUAGGIO A BLOCCHI

**Studenti/Classe**: S2B

**Formatore**: *Massi*

**Tutor**: *Pomili*

**Descrizione**: Il corso è un'introduzione alle "grandi idee" della crittografia. L'approccio didattico procede "per scoperta" con attività che si susseguono con una successione di crittosistemi (dai classici ai più moderni). Lo studente può sperimentare caratteristiche di ciascun crittosistema, realizzado possibili procedure di attacco e valutandone i limiti. Dalle debolezze di un sistema lo studente dovrebbe avere necessità di scoprire il crittosistema successivo, che avrà superato i limiti del sistema precedente ma che potrà presentare altri aspetti limitanti. L'impianto generale del corso si basa su M. Lodi - M. Sbaraglia - S. Martini, "PROGRAMMARE PER IMPARARE LA CRITTOGRAFIA AL LICEO MATEMATICO". I crittosistemi, la loro analisi ed le possibili procedure di attacco sono realizzate in Snap! (un linguaggio di programmazione visuale a blocchi) da Michael Lodi (Università di Bologna).

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|mer 05/02/2025 |1|2|mer 05/02/2025 |4|
|3|mer 12/02/2025 |4|4|mer 19/02/2025 |4|
|5|mer 05/03/2025 |4|6|mer 16/04/2025 |4|
|7|mer 23/04/2025 |4|8|mer 30/04/2025 |4|
|9|mer 07/05/2025 |4|10|mer 14/05/2025 |4|



## LABORATORIO DI MATEMATICA 2

**Studenti/Classe**: LicMat

**Formatore**: *Massi*

**Tutor**: *Marrucchiello*

**Descrizione**: Il corso mira a sviluppare il pensiero ricorsivo attraverso lo sviluppo dell'aritmetica. Tra induzione e ricorsione saranno definiti l'insieme dei numeri naturali e, con le sole funzioni primitive di riconoscimento dello zero, successore e predecessore, saranno definiti alcuni operatori di confronto e operazioni di addizione, sottrazione, moltiplicazione e divisione intera. Lo stesso modo di ragionare su problemi numerici può essere applicato alla risoluzione di tutti problemi che l'informatica può risolvere. Sono questi ragionamenti il vero oggetto del corso.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|ven 31/01/2025 |5|2|ven 31/01/2025 |6|
|3|ven 07/02/2025 |5|4|ven 07/02/2025 |6|
|5|ven 14/02/2025 |5|6|ven 14/02/2025 |6|
|7|ven 21/02/2025 |5|8|ven 21/02/2025 |6|
|9|ven 07/03/2025 |5|10|ven 07/03/2025 |6|



## PHYSICAL COMPUTING

**Studenti/Classe**: E2B

**Formatore**: *Massi*

**Tutor**: *Sciamanna*

**Descrizione**: Il percorso vuole fornire un'introduzione al Physical computing e usando Micro:bit, Raspberry Pi e Arduino. L'uso delle schede elettroniche programmabili permette l'interazione con il mondo reale e rende visibile l'effetto dell'esecuzione del programma che lo studente andrà a realizzare. Obiettivi del corso sono la 

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|lun 10/03/2025 |2|2|mer 12/03/2025 |4|
|3|lun 17/03/2025 |2|4|mer 19/03/2025 |4|
|5|lun 24/03/2025 |2|6|mer 26/03/2025 |4|
|7|lun 31/03/2025 |2|8|mer 02/04/2025 |4|
|9|lun 07/04/2025 |2|10|mer 09/04/2025 |4|



## RICERCA OPERATIVA

**Studenti/Classe**: E5ASIA

**Formatore**: *Marinelli*

**Tutor**: *Massi*

**Descrizione**: Il laboratorio introduce, per mezzo di attività laboratoriali, le problematiche affrontate della Ricerca Operativa, una disciplina che si avvale di metodi scientifici, matematici e informatici per risolvere problemi complessi che si presentano in svariati contesti, dalla logistica all'economia, passando per l'ingegneria e la medicina.

**Calendario**

| Incontro | Data | Ora | Incontro | Data | Ora |
|--:|:-:|:-:|--:|:-:|:-:|
|1|lun 03/02/2025 |1|2|lun 03/02/2025 |2|
|3|lun 10/02/2025 |1|4|lun 10/02/2025 |2|
|5|lun 17/02/2025 |1|6|lun 17/02/2025 |2|
|7|lun 24/02/2025 |1|8|lun 24/02/2025 |2|
|9|lun 03/03/2025 |1|10|lun 03/03/2025 |2|





In [12]:
report.clear()
# Orario formatori
report.addTitle("Orario del personale")
for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    staffLessonsRep = Report()
    staffLessonsRep.addSubTitle(f"{p} - Lezioni")
    staffLessonsRep.addLessonsByStaff(p, con)
    staffLessonsRep.saveToFile("./gen_docs/lessons/" + p.lower() + ".md")
    report.append(staffLessonsRep.markdown())

report.display()

# Orario del personale

## Lorenzoni - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mar 04/02/2025|2|CryptoWithBlocks|1|S4G|
|2|mar 11/02/2025|2|CryptoWithBlocks|2|S4G|
|3|mar 18/02/2025|2|CryptoWithBlocks|3|S4G|
|4|mar 25/02/2025|2|CryptoWithBlocks|4|S4G|
|5|mar 04/03/2025|2|CryptoWithBlocks|5|S4G|
|6|mar 11/03/2025|2|CryptoWithBlocks|6|S4G|
|7|mar 18/03/2025|2|CryptoWithBlocks|7|S4G|
|8|mar 25/03/2025|2|CryptoWithBlocks|8|S4G|
|9|mar 01/04/2025|2|CryptoWithBlocks|9|S4G|
|10|mar 08/04/2025|2|CryptoWithBlocks|10|S4G|



## Massi - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mer 11/12/2024|3|RelAx|1|S5E|
|2|mer 11/12/2024|4|RelAx|1|S4F|
|3|ven 13/12/2024|5|RelAx|1|S5B+S5D|
|4|lun 16/12/2024|1|RelAx|2|S4F|
|5|lun 16/12/2024|2|RelAx|2|S5E|
|6|mer 18/12/2024|3|RelAx|3|S5E|
|7|mer 18/12/2024|4|RelAx|3|S4F|
|8|ven 20/12/2024|5|RelAx|2|S5B+S5D|
|9|mer 08/01/2025|3|RelAx|4|S5E|
|10|mer 08/01/2025|4|RelAx|4|S4F|
|11|ven 10/01/2025|5|RelAx|3|S5B+S5D|
|12|lun 13/01/2025|1|RelAx|5|S4F|
|13|lun 13/01/2025|2|RelAx|5|S5E|
|14|mer 15/01/2025|3|RelAx|6|S5E|
|15|mer 15/01/2025|4|RelAx|6|S4F|
|16|ven 17/01/2025|5|RelAx|4|S5B+S5D|
|17|lun 20/01/2025|1|RelAx|7|S4F|
|18|lun 20/01/2025|2|RelAx|7|S5E|
|19|mer 22/01/2025|3|RelAx|8|S5E|
|20|mer 22/01/2025|4|RelAx|8|S4F|
|21|ven 24/01/2025|5|RelAx|5|S5B+S5D|
|22|lun 27/01/2025|1|RelAx|9|S4F|
|23|lun 27/01/2025|2|RelAx|9|S5E|
|24|mer 29/01/2025|3|RelAx|10|S5E|
|25|mer 29/01/2025|4|RelAx|10|S4F|
|26|ven 31/01/2025|5|LabMat2|1|LicMat|
|27|ven 31/01/2025|6|LabMat2|2|LicMat|
|28|lun 03/02/2025|1|RicOp|1|E5ASIA|
|29|lun 03/02/2025|2|RicOp|2|E5ASIA|
|30|mer 05/02/2025|1|CryptoWithBlocks|1|S2B|
|31|mer 05/02/2025|2|RelAx|1|S4E|
|32|mer 05/02/2025|4|CryptoWithBlocks|2|S2B|
|33|ven 07/02/2025|5|LabMat2|3|LicMat|
|34|ven 07/02/2025|6|LabMat2|4|LicMat|
|35|lun 10/02/2025|1|RicOp|3|E5ASIA|
|36|lun 10/02/2025|2|RicOp|4|E5ASIA|
|37|mer 12/02/2025|2|RelAx|2|S4E|
|38|mer 12/02/2025|4|CryptoWithBlocks|3|S2B|
|39|ven 14/02/2025|5|LabMat2|5|LicMat|
|40|ven 14/02/2025|6|LabMat2|6|LicMat|
|41|lun 17/02/2025|1|RicOp|5|E5ASIA|
|42|lun 17/02/2025|2|RicOp|6|E5ASIA|
|43|mer 19/02/2025|2|RelAx|3|S4E|
|44|mer 19/02/2025|4|CryptoWithBlocks|4|S2B|
|45|ven 21/02/2025|5|LabMat2|7|LicMat|
|46|ven 21/02/2025|6|LabMat2|8|LicMat|
|47|lun 24/02/2025|1|RicOp|7|E5ASIA|
|48|lun 24/02/2025|2|RicOp|8|E5ASIA|
|49|mer 26/02/2025|2|RelAx|4|S4E|
|50|lun 03/03/2025|1|RicOp|9|E5ASIA|
|51|lun 03/03/2025|2|RicOp|10|E5ASIA|
|52|mer 05/03/2025|2|RelAx|5|S4E|
|53|mer 05/03/2025|4|CryptoWithBlocks|5|S2B|
|54|ven 07/03/2025|5|LabMat2|9|LicMat|
|55|ven 07/03/2025|6|LabMat2|10|LicMat|
|56|lun 10/03/2025|2|PhyComp|1|E2B|
|57|mer 12/03/2025|2|RelAx|6|S4E|
|58|mer 12/03/2025|4|PhyComp|2|E2B|
|59|ven 14/03/2025|5|RelAx|6|S5B+S5D|
|60|lun 17/03/2025|2|PhyComp|3|E2B|
|61|mer 19/03/2025|2|RelAx|7|S4E|
|62|mer 19/03/2025|4|PhyComp|4|E2B|
|63|ven 21/03/2025|5|RelAx|7|S5B+S5D|
|64|lun 24/03/2025|2|PhyComp|5|E2B|
|65|mer 26/03/2025|2|RelAx|8|S4E|
|66|mer 26/03/2025|4|PhyComp|6|E2B|
|67|ven 28/03/2025|5|RelAx|8|S5B+S5D|
|68|lun 31/03/2025|2|PhyComp|7|E2B|
|69|mer 02/04/2025|2|RelAx|9|S4E|
|70|mer 02/04/2025|4|PhyComp|8|E2B|
|71|ven 04/04/2025|5|RelAx|9|S5B+S5D|
|72|lun 07/04/2025|2|PhyComp|9|E2B|
|73|mer 09/04/2025|2|RelAx|10|S4E|
|74|mer 09/04/2025|4|PhyComp|10|E2B|
|75|ven 11/04/2025|5|RelAx|10|S5B+S5D|
|76|mer 16/04/2025|4|CryptoWithBlocks|6|S2B|
|77|mer 23/04/2025|4|CryptoWithBlocks|7|S2B|
|78|mer 30/04/2025|4|CryptoWithBlocks|8|S2B|
|79|mer 07/05/2025|4|CryptoWithBlocks|9|S2B|
|80|mer 14/05/2025|4|CryptoWithBlocks|10|S2B|



## Pomili - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|ven 13/12/2024|5|RelAx|1|S5B+S5D|
|2|ven 20/12/2024|5|RelAx|2|S5B+S5D|
|3|ven 10/01/2025|5|RelAx|3|S5B+S5D|
|4|ven 17/01/2025|5|RelAx|4|S5B+S5D|
|5|ven 24/01/2025|5|RelAx|5|S5B+S5D|
|6|mer 05/02/2025|1|CryptoWithBlocks|1|S2B|
|7|mer 05/02/2025|2|RelAx|1|S4E|
|8|mer 05/02/2025|4|CryptoWithBlocks|2|S2B|
|9|mer 12/02/2025|2|RelAx|2|S4E|
|10|mer 12/02/2025|4|CryptoWithBlocks|3|S2B|
|11|mer 19/02/2025|2|RelAx|3|S4E|
|12|mer 19/02/2025|4|CryptoWithBlocks|4|S2B|
|13|mer 26/02/2025|2|RelAx|4|S4E|
|14|mer 05/03/2025|2|RelAx|5|S4E|
|15|mer 05/03/2025|4|CryptoWithBlocks|5|S2B|
|16|mer 12/03/2025|2|RelAx|6|S4E|
|17|ven 14/03/2025|5|RelAx|6|S5B+S5D|
|18|mer 19/03/2025|2|RelAx|7|S4E|
|19|ven 21/03/2025|5|RelAx|7|S5B+S5D|
|20|mer 26/03/2025|2|RelAx|8|S4E|
|21|ven 28/03/2025|5|RelAx|8|S5B+S5D|
|22|mer 02/04/2025|2|RelAx|9|S4E|
|23|ven 04/04/2025|5|RelAx|9|S5B+S5D|
|24|mer 09/04/2025|2|RelAx|10|S4E|
|25|ven 11/04/2025|5|RelAx|10|S5B+S5D|
|26|mer 16/04/2025|4|CryptoWithBlocks|6|S2B|
|27|mer 23/04/2025|4|CryptoWithBlocks|7|S2B|
|28|mer 30/04/2025|4|CryptoWithBlocks|8|S2B|
|29|mer 07/05/2025|4|CryptoWithBlocks|9|S2B|
|30|mer 14/05/2025|4|CryptoWithBlocks|10|S2B|



## Sciamanna - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mer 11/12/2024|3|RelAx|1|S5E|
|2|mer 11/12/2024|4|RelAx|1|S4F|
|3|lun 16/12/2024|1|RelAx|2|S4F|
|4|lun 16/12/2024|2|RelAx|2|S5E|
|5|mer 18/12/2024|3|RelAx|3|S5E|
|6|mer 18/12/2024|4|RelAx|3|S4F|
|7|mer 08/01/2025|3|RelAx|4|S5E|
|8|mer 08/01/2025|4|RelAx|4|S4F|
|9|lun 13/01/2025|1|RelAx|5|S4F|
|10|lun 13/01/2025|2|RelAx|5|S5E|
|11|mer 15/01/2025|3|RelAx|6|S5E|
|12|mer 15/01/2025|4|RelAx|6|S4F|
|13|lun 20/01/2025|1|RelAx|7|S4F|
|14|lun 20/01/2025|2|RelAx|7|S5E|
|15|mer 22/01/2025|3|RelAx|8|S5E|
|16|mer 22/01/2025|4|RelAx|8|S4F|
|17|lun 27/01/2025|1|RelAx|9|S4F|
|18|lun 27/01/2025|2|RelAx|9|S5E|
|19|mer 29/01/2025|3|RelAx|10|S5E|
|20|mer 29/01/2025|4|RelAx|10|S4F|
|21|mer 29/01/2025|5|CyberSec|1|S3B|
|22|mer 29/01/2025|6|CyberSec|1|S4B|
|23|ven 31/01/2025|4|CyberSec|2|S4B|
|24|ven 31/01/2025|6|CyberSec|2|S3B|
|25|mer 05/02/2025|1|CryptoWithBlocks|1|S1B|
|26|mer 05/02/2025|5|CyberSec|3|S3B|
|27|mer 05/02/2025|6|CyberSec|3|S4B|
|28|ven 07/02/2025|4|CyberSec|4|S4B|
|29|ven 07/02/2025|6|CyberSec|4|S3B|
|30|sab 08/02/2025|4|CryptoWithBlocks|2|S1B|
|31|mer 12/02/2025|1|CryptoWithBlocks|3|S1B|
|32|mer 12/02/2025|5|CyberSec|5|S3B|
|33|mer 12/02/2025|6|CyberSec|5|S4B|
|34|sab 15/02/2025|4|CryptoWithBlocks|4|S1B|
|35|mer 19/02/2025|1|CryptoWithBlocks|5|S1B|
|36|mer 19/02/2025|5|CyberSec|6|S3B|
|37|mer 19/02/2025|6|CyberSec|6|S4B|
|38|ven 21/02/2025|4|CyberSec|7|S4B|
|39|ven 21/02/2025|6|CyberSec|7|S3B|
|40|sab 22/02/2025|4|CryptoWithBlocks|6|S1B|
|41|mer 26/02/2025|1|CryptoWithBlocks|7|S1B|
|42|mer 26/02/2025|6|CyberSec|8|S4B|
|43|ven 28/02/2025|4|CyberSec|9|S4B|
|44|sab 01/03/2025|4|CryptoWithBlocks|8|S1B|
|45|mer 05/03/2025|1|CryptoWithBlocks|9|S1B|
|46|mer 05/03/2025|5|CyberSec|8|S3B|
|47|mer 05/03/2025|6|CyberSec|10|S4B|
|48|ven 07/03/2025|4|CyberSec|11|S4B|
|49|ven 07/03/2025|6|CyberSec|9|S3B|
|50|sab 08/03/2025|4|CryptoWithBlocks|10|S1B|
|51|lun 10/03/2025|2|PhyComp|1|E2B|
|52|mer 12/03/2025|4|PhyComp|2|E2B|
|53|mer 12/03/2025|5|CyberSec|10|S3B|
|54|mer 12/03/2025|6|CyberSec|12|S4B|
|55|ven 14/03/2025|4|CyberSec|13|S4B|
|56|ven 14/03/2025|6|CyberSec|11|S3B|
|57|lun 17/03/2025|2|PhyComp|3|E2B|
|58|mer 19/03/2025|4|PhyComp|4|E2B|
|59|mer 19/03/2025|5|CyberSec|12|S3B|
|60|mer 19/03/2025|6|CyberSec|14|S4B|
|61|ven 21/03/2025|4|CyberSec|15|S4B|
|62|ven 21/03/2025|6|CyberSec|13|S3B|
|63|lun 24/03/2025|2|PhyComp|5|E2B|
|64|mer 26/03/2025|4|PhyComp|6|E2B|
|65|mer 26/03/2025|5|CyberSec|14|S3B|
|66|mer 26/03/2025|6|CyberSec|16|S4B|
|67|ven 28/03/2025|4|CyberSec|17|S4B|
|68|ven 28/03/2025|6|CyberSec|15|S3B|
|69|lun 31/03/2025|2|PhyComp|7|E2B|
|70|mer 02/04/2025|4|PhyComp|8|E2B|
|71|mer 02/04/2025|5|CyberSec|16|S3B|
|72|mer 02/04/2025|6|CyberSec|18|S4B|
|73|ven 04/04/2025|4|CyberSec|19|S4B|
|74|ven 04/04/2025|6|CyberSec|17|S3B|
|75|lun 07/04/2025|2|PhyComp|9|E2B|
|76|mer 09/04/2025|4|PhyComp|10|E2B|
|77|mer 09/04/2025|5|CyberSec|18|S3B|
|78|mer 09/04/2025|6|CyberSec|20|S4B|
|79|ven 11/04/2025|6|CyberSec|19|S3B|
|80|mer 16/04/2025|5|CyberSec|20|S3B|



## Taccari - Lezioni

|Progr.| Data | Ora | Corso | Lezione | Classe |
|--:|:-:|:-:|:-:|--:|:-:|
|1|mer 29/01/2025|5|CyberSec|1|S3B|
|2|mer 29/01/2025|6|CyberSec|1|S4B|
|3|ven 31/01/2025|4|CyberSec|2|S4B|
|4|ven 31/01/2025|6|CyberSec|2|S3B|
|5|mar 04/02/2025|2|CryptoWithBlocks|1|S4G|
|6|mer 05/02/2025|1|CryptoWithBlocks|1|S1B|
|7|mer 05/02/2025|5|CyberSec|3|S3B|
|8|mer 05/02/2025|6|CyberSec|3|S4B|
|9|ven 07/02/2025|4|CyberSec|4|S4B|
|10|ven 07/02/2025|6|CyberSec|4|S3B|
|11|sab 08/02/2025|4|CryptoWithBlocks|2|S1B|
|12|mar 11/02/2025|2|CryptoWithBlocks|2|S4G|
|13|mer 12/02/2025|1|CryptoWithBlocks|3|S1B|
|14|mer 12/02/2025|5|CyberSec|5|S3B|
|15|mer 12/02/2025|6|CyberSec|5|S4B|
|16|sab 15/02/2025|4|CryptoWithBlocks|4|S1B|
|17|mar 18/02/2025|2|CryptoWithBlocks|3|S4G|
|18|mer 19/02/2025|1|CryptoWithBlocks|5|S1B|
|19|mer 19/02/2025|5|CyberSec|6|S3B|
|20|mer 19/02/2025|6|CyberSec|6|S4B|
|21|ven 21/02/2025|4|CyberSec|7|S4B|
|22|ven 21/02/2025|6|CyberSec|7|S3B|
|23|sab 22/02/2025|4|CryptoWithBlocks|6|S1B|
|24|mar 25/02/2025|2|CryptoWithBlocks|4|S4G|
|25|mer 26/02/2025|1|CryptoWithBlocks|7|S1B|
|26|mer 26/02/2025|6|CyberSec|8|S4B|
|27|ven 28/02/2025|4|CyberSec|9|S4B|
|28|sab 01/03/2025|4|CryptoWithBlocks|8|S1B|
|29|mar 04/03/2025|2|CryptoWithBlocks|5|S4G|
|30|mer 05/03/2025|1|CryptoWithBlocks|9|S1B|
|31|mer 05/03/2025|5|CyberSec|8|S3B|
|32|mer 05/03/2025|6|CyberSec|10|S4B|
|33|ven 07/03/2025|4|CyberSec|11|S4B|
|34|ven 07/03/2025|6|CyberSec|9|S3B|
|35|sab 08/03/2025|4|CryptoWithBlocks|10|S1B|
|36|mar 11/03/2025|2|CryptoWithBlocks|6|S4G|
|37|mer 12/03/2025|5|CyberSec|10|S3B|
|38|mer 12/03/2025|6|CyberSec|12|S4B|
|39|ven 14/03/2025|4|CyberSec|13|S4B|
|40|ven 14/03/2025|6|CyberSec|11|S3B|
|41|mar 18/03/2025|2|CryptoWithBlocks|7|S4G|
|42|mer 19/03/2025|5|CyberSec|12|S3B|
|43|mer 19/03/2025|6|CyberSec|14|S4B|
|44|ven 21/03/2025|4|CyberSec|15|S4B|
|45|ven 21/03/2025|6|CyberSec|13|S3B|
|46|mar 25/03/2025|2|CryptoWithBlocks|8|S4G|
|47|mer 26/03/2025|5|CyberSec|14|S3B|
|48|mer 26/03/2025|6|CyberSec|16|S4B|
|49|ven 28/03/2025|4|CyberSec|17|S4B|
|50|ven 28/03/2025|6|CyberSec|15|S3B|
|51|mar 01/04/2025|2|CryptoWithBlocks|9|S4G|
|52|mer 02/04/2025|5|CyberSec|16|S3B|
|53|mer 02/04/2025|6|CyberSec|18|S4B|
|54|ven 04/04/2025|4|CyberSec|19|S4B|
|55|ven 04/04/2025|6|CyberSec|17|S3B|
|56|mar 08/04/2025|2|CryptoWithBlocks|10|S4G|
|57|mer 09/04/2025|5|CyberSec|18|S3B|
|58|mer 09/04/2025|6|CyberSec|20|S4B|
|59|ven 11/04/2025|6|CyberSec|19|S3B|
|60|mer 16/04/2025|5|CyberSec|20|S3B|





In [13]:
report.clear()
# Impegni giornalieri
report.addTitle("Orario del personale")
for p in ['Lorenzoni', 'Massi', 'Pomili', 'Sciamanna', 'Taccari']:
    schedRep = Report()
    schedRep.addSubTitle(f"{p} - Giorni")
    schedRep.addDailySchedule(p, con)
    schedRep.saveToFile("./gen_docs/schedule/" + p.lower() + ".md")
    report.append(schedRep.markdown())

report.display()

# Orario del personale

## Lorenzoni - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mar 04/02/2025|2|CryptoWithBlocks|
|2|mar 11/02/2025|2|CryptoWithBlocks|
|3|mar 18/02/2025|2|CryptoWithBlocks|
|4|mar 25/02/2025|2|CryptoWithBlocks|
|5|mar 04/03/2025|2|CryptoWithBlocks|
|6|mar 11/03/2025|2|CryptoWithBlocks|
|7|mar 18/03/2025|2|CryptoWithBlocks|
|8|mar 25/03/2025|2|CryptoWithBlocks|
|9|mar 01/04/2025|2|CryptoWithBlocks|
|10|mar 08/04/2025|2|CryptoWithBlocks|



## Massi - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mer 11/12/2024|3,4|RelAx|
|2|ven 13/12/2024|5|RelAx|
|3|lun 16/12/2024|1,2|RelAx|
|4|mer 18/12/2024|3,4|RelAx|
|5|ven 20/12/2024|5|RelAx|
|6|mer 08/01/2025|3,4|RelAx|
|7|ven 10/01/2025|5|RelAx|
|8|lun 13/01/2025|1,2|RelAx|
|9|mer 15/01/2025|3,4|RelAx|
|10|ven 17/01/2025|5|RelAx|
|11|lun 20/01/2025|1,2|RelAx|
|12|mer 22/01/2025|3,4|RelAx|
|13|ven 24/01/2025|5|RelAx|
|14|lun 27/01/2025|1,2|RelAx|
|15|mer 29/01/2025|3,4|RelAx|
|16|ven 31/01/2025|5,6|LabMat2|
|17|lun 03/02/2025|1,2|RicOp|
|18|mer 05/02/2025|1,2,4|CryptoWithBlocks,RelAx|
|19|ven 07/02/2025|5,6|LabMat2|
|20|lun 10/02/2025|1,2|RicOp|
|21|mer 12/02/2025|2,4|CryptoWithBlocks,RelAx|
|22|ven 14/02/2025|5,6|LabMat2|
|23|lun 17/02/2025|1,2|RicOp|
|24|mer 19/02/2025|2,4|CryptoWithBlocks,RelAx|
|25|ven 21/02/2025|5,6|LabMat2|
|26|lun 24/02/2025|1,2|RicOp|
|27|mer 26/02/2025|2|RelAx|
|28|lun 03/03/2025|1,2|RicOp|
|29|mer 05/03/2025|2,4|CryptoWithBlocks,RelAx|
|30|ven 07/03/2025|5,6|LabMat2|
|31|lun 10/03/2025|2|PhyComp|
|32|mer 12/03/2025|2,4|PhyComp,RelAx|
|33|ven 14/03/2025|5|RelAx|
|34|lun 17/03/2025|2|PhyComp|
|35|mer 19/03/2025|2,4|PhyComp,RelAx|
|36|ven 21/03/2025|5|RelAx|
|37|lun 24/03/2025|2|PhyComp|
|38|mer 26/03/2025|2,4|PhyComp,RelAx|
|39|ven 28/03/2025|5|RelAx|
|40|lun 31/03/2025|2|PhyComp|
|41|mer 02/04/2025|2,4|PhyComp,RelAx|
|42|ven 04/04/2025|5|RelAx|
|43|lun 07/04/2025|2|PhyComp|
|44|mer 09/04/2025|2,4|PhyComp,RelAx|
|45|ven 11/04/2025|5|RelAx|
|46|mer 16/04/2025|4|CryptoWithBlocks|
|47|mer 23/04/2025|4|CryptoWithBlocks|
|48|mer 30/04/2025|4|CryptoWithBlocks|
|49|mer 07/05/2025|4|CryptoWithBlocks|
|50|mer 14/05/2025|4|CryptoWithBlocks|



## Pomili - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|ven 13/12/2024|5|RelAx|
|2|ven 20/12/2024|5|RelAx|
|3|ven 10/01/2025|5|RelAx|
|4|ven 17/01/2025|5|RelAx|
|5|ven 24/01/2025|5|RelAx|
|6|mer 05/02/2025|1,2,4|CryptoWithBlocks,RelAx|
|7|mer 12/02/2025|2,4|CryptoWithBlocks,RelAx|
|8|mer 19/02/2025|2,4|CryptoWithBlocks,RelAx|
|9|mer 26/02/2025|2|RelAx|
|10|mer 05/03/2025|2,4|CryptoWithBlocks,RelAx|
|11|mer 12/03/2025|2|RelAx|
|12|ven 14/03/2025|5|RelAx|
|13|mer 19/03/2025|2|RelAx|
|14|ven 21/03/2025|5|RelAx|
|15|mer 26/03/2025|2|RelAx|
|16|ven 28/03/2025|5|RelAx|
|17|mer 02/04/2025|2|RelAx|
|18|ven 04/04/2025|5|RelAx|
|19|mer 09/04/2025|2|RelAx|
|20|ven 11/04/2025|5|RelAx|
|21|mer 16/04/2025|4|CryptoWithBlocks|
|22|mer 23/04/2025|4|CryptoWithBlocks|
|23|mer 30/04/2025|4|CryptoWithBlocks|
|24|mer 07/05/2025|4|CryptoWithBlocks|
|25|mer 14/05/2025|4|CryptoWithBlocks|



## Sciamanna - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mer 11/12/2024|3,4|RelAx|
|2|lun 16/12/2024|1,2|RelAx|
|3|mer 18/12/2024|3,4|RelAx|
|4|mer 08/01/2025|3,4|RelAx|
|5|lun 13/01/2025|1,2|RelAx|
|6|mer 15/01/2025|3,4|RelAx|
|7|lun 20/01/2025|1,2|RelAx|
|8|mer 22/01/2025|3,4|RelAx|
|9|lun 27/01/2025|1,2|RelAx|
|10|mer 29/01/2025|3,4,5,6|CyberSec,RelAx|
|11|ven 31/01/2025|4,6|CyberSec|
|12|mer 05/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|13|ven 07/02/2025|4,6|CyberSec|
|14|sab 08/02/2025|4|CryptoWithBlocks|
|15|mer 12/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|16|sab 15/02/2025|4|CryptoWithBlocks|
|17|mer 19/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|18|ven 21/02/2025|4,6|CyberSec|
|19|sab 22/02/2025|4|CryptoWithBlocks|
|20|mer 26/02/2025|1,6|CryptoWithBlocks,CyberSec|
|21|ven 28/02/2025|4|CyberSec|
|22|sab 01/03/2025|4|CryptoWithBlocks|
|23|mer 05/03/2025|1,5,6|CryptoWithBlocks,CyberSec|
|24|ven 07/03/2025|4,6|CyberSec|
|25|sab 08/03/2025|4|CryptoWithBlocks|
|26|lun 10/03/2025|2|PhyComp|
|27|mer 12/03/2025|4,5,6|CyberSec,PhyComp|
|28|ven 14/03/2025|4,6|CyberSec|
|29|lun 17/03/2025|2|PhyComp|
|30|mer 19/03/2025|4,5,6|CyberSec,PhyComp|
|31|ven 21/03/2025|4,6|CyberSec|
|32|lun 24/03/2025|2|PhyComp|
|33|mer 26/03/2025|4,5,6|CyberSec,PhyComp|
|34|ven 28/03/2025|4,6|CyberSec|
|35|lun 31/03/2025|2|PhyComp|
|36|mer 02/04/2025|4,5,6|CyberSec,PhyComp|
|37|ven 04/04/2025|4,6|CyberSec|
|38|lun 07/04/2025|2|PhyComp|
|39|mer 09/04/2025|4,5,6|CyberSec,PhyComp|
|40|ven 11/04/2025|6|CyberSec|
|41|mer 16/04/2025|5|CyberSec|



## Taccari - Giorni

|Giornata| Data | Ore | Corsi |
|--:|:-:|:-:|:-:|
|1|mer 29/01/2025|5,6|CyberSec|
|2|ven 31/01/2025|4,6|CyberSec|
|3|mar 04/02/2025|2|CryptoWithBlocks|
|4|mer 05/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|5|ven 07/02/2025|4,6|CyberSec|
|6|sab 08/02/2025|4|CryptoWithBlocks|
|7|mar 11/02/2025|2|CryptoWithBlocks|
|8|mer 12/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|9|sab 15/02/2025|4|CryptoWithBlocks|
|10|mar 18/02/2025|2|CryptoWithBlocks|
|11|mer 19/02/2025|1,5,6|CryptoWithBlocks,CyberSec|
|12|ven 21/02/2025|4,6|CyberSec|
|13|sab 22/02/2025|4|CryptoWithBlocks|
|14|mar 25/02/2025|2|CryptoWithBlocks|
|15|mer 26/02/2025|1,6|CryptoWithBlocks,CyberSec|
|16|ven 28/02/2025|4|CyberSec|
|17|sab 01/03/2025|4|CryptoWithBlocks|
|18|mar 04/03/2025|2|CryptoWithBlocks|
|19|mer 05/03/2025|1,5,6|CryptoWithBlocks,CyberSec|
|20|ven 07/03/2025|4,6|CyberSec|
|21|sab 08/03/2025|4|CryptoWithBlocks|
|22|mar 11/03/2025|2|CryptoWithBlocks|
|23|mer 12/03/2025|5,6|CyberSec|
|24|ven 14/03/2025|4,6|CyberSec|
|25|mar 18/03/2025|2|CryptoWithBlocks|
|26|mer 19/03/2025|5,6|CyberSec|
|27|ven 21/03/2025|4,6|CyberSec|
|28|mar 25/03/2025|2|CryptoWithBlocks|
|29|mer 26/03/2025|5,6|CyberSec|
|30|ven 28/03/2025|4,6|CyberSec|
|31|mar 01/04/2025|2|CryptoWithBlocks|
|32|mer 02/04/2025|5,6|CyberSec|
|33|ven 04/04/2025|4,6|CyberSec|
|34|mar 08/04/2025|2|CryptoWithBlocks|
|35|mer 09/04/2025|5,6|CyberSec|
|36|ven 11/04/2025|6|CyberSec|
|37|mer 16/04/2025|5|CyberSec|





In [14]:
con.close()